# Create Amazon Redshift Cluster

Amazon Redshift is a fully managed data warehouse which allows you to run complex analytic queries against petabytes of structured data. Your queries are distributed and parallelized across multiple physical resources, and you can easily scale your Amazon Redshift environment up and down depending on your business needs. 

_Note:  This notebook requires that you are running this SageMaker Notebook Instance in a VPC with access to the Redshift cluster._

## Setup IAM Access To Read From S3 and Athena

AWS Identity and Access Management (IAM) is a service that helps you to manage access to AWS resources. IAM controls who is authenticated and authorized to use resources. 

You can create individual IAM users for people accessing your AWS account. Each user will have a unique set of security credentials. You can also assign IAM users to IAM groups with defined access permissions (i.e. for specific job functions) and the IAM users inherit those permissions. 

A more preferred way to delegate access permissions is via IAM roles. In contrast to an IAM user which is uniquely associated with one person, a role can be assumed by anyone who needs it, and provides you with only temporary security credentials for the duration of the role session. AWS Service Roles control which actions a service can perform on your behalf. 

Access permissions are defined using IAM policies. It’s a standard security best practice to only grant least privilege, in other words- only grant the permissions required to perform a task. 



In [1]:
import json
import boto3
from botocore.exceptions import ClientError
from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)


iam = boto3.client('iam', config=config)
sts = boto3.client('sts')
accountID = sts.get_caller_identity()["Account"]  
redshift = boto3.client('redshift')
sm = boto3.client('sagemaker')
ec2 = boto3.client('ec2')

### Variables

In [ ]:
# Redshift configuration parameters
redshift_cluster_identifier = 'bankdemo'
database_name = 'bankdemo'
cluster_type = 'single-node' # or multi-node

master_user_name = 'bankdemo'
master_user_pw = 'k8aPdBXXbVwwshY'

# Note that only some Instance Types support Redshift Query Editor 
# (https://docs.aws.amazon.com/redshift/latest/mgmt/query-editor.html)
node_type = 'dc2.large'
# number_nodes = '1' 

# Set the security group ID if there are more than one
security_group_id = 'sg-044c6f07030b3be91'

### Create AssumeRolePolicyDocument

In [2]:
role = f"arn:aws:iam::{accountID}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole"
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "AWS": role,
        "Service": ["sagemaker.amazonaws.com", "redshift.amazonaws.com"]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

assume_role_policy_doc

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'AWS': 'arn:aws:iam::138604873012:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole',
    'Service': ['sagemaker.amazonaws.com', 'redshift.amazonaws.com']},
   'Action': 'sts:AssumeRole'}]}

### Create Role

In [3]:
iam_redshift_role_name = 'BankDemo'

In [4]:
try:
    iam_role_redshift = iam.create_role(
        RoleName=iam_redshift_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='Bank Redshift Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


#### Get the Role ARN

In [5]:
role = iam.get_role(RoleName=iam_redshift_role_name)
iam_role_redshift_arn = role['Role']['Arn']
print(iam_role_redshift_arn)

arn:aws:iam::138604873012:role/BankDemo


## Create Self-Managed Policies

### Define Policies

#### arn:aws:iam::aws:policy/AmazonS3FullAccess

In [6]:
my_redshift_to_s3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": "*"
        }
    ]
}

#### arn:aws:iam::aws:policy/AmazonAthenaFullAccess

In [7]:
my_redshift_to_athena = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "athena:*"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload",
                "s3:CreateBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::aws-athena-query-results-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::athena-examples*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:ListAllMyBuckets"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "sns:ListTopics",
                "sns:GetTopicAttributes"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricAlarm",
                "cloudwatch:DescribeAlarms",
                "cloudwatch:DeleteAlarms"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "lakeformation:GetDataAccess"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

### Create Policy Objects

In [8]:
policyName = 'BankDemo_RedshiftPolicyToS3'
try:
    policy_redshift_s3 = iam.create_policy(
      PolicyName=policyName,
      PolicyDocument=json.dumps(my_redshift_to_s3)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

Policy already exists


In [9]:
# Get ARN
account_id = sts.get_caller_identity()['Account']
policy_redshift_s3_arn = f'arn:aws:iam::{account_id}:policy/{policyName}'
print(policy_redshift_s3_arn)


arn:aws:iam::138604873012:policy/BankDemo_RedshiftPolicyToS3


In [10]:
policyName = 'BankDemo_RedshiftPolicyToAthena'
try:
    policy_redshift_athena = iam.create_policy(
      PolicyName=policyName,
      PolicyDocument=json.dumps(my_redshift_to_athena)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

Policy already exists


In [11]:
# Get ARN
account_id = sts.get_caller_identity()['Account']
policy_redshift_athena_arn = f'arn:aws:iam::{account_id}:policy/{policyName}'
print(policy_redshift_athena_arn)


arn:aws:iam::138604873012:policy/BankDemo_RedshiftPolicyToAthena


### Attach Policies To Role

In [12]:
# Attach RedshiftPolicyToAthena policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_athena_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Attach RedshiftPolicyToS3 policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_s3_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)
        

In [13]:
           
# Attach SecretsManager policy
try:
    response = iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/SecretsManagerReadWrite',
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)
        
# Attach RedShift policy
try:
    response = iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/AmazonRedshiftFullAccess',
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Attach SageMaker policy
try:
    response = iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess',
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

### Get Security Group ID 

* Make sure the Redshift VPC is the same this notebook is running within
* Make sure the VPC has the following 2 properties enabled
 *     DNS resolution = Enabled
 *     DNS hostnames = Enabled
* This allows private, internal access to Redshift from this SageMaker notebook using the fully qualified endpoint name.

In [ ]:
# for sagemaker studio
# try:
#     domain_id = sm.list_domains()['Domains'][0]['DomainId'] #['NotebookInstances'][0]['NotebookInstanceName']
#     describe_domain_response = sm.describe_domain(DomainId=domain_id)
#     vpc_id = describe_domain_response['VpcId']
#     security_groups = ec2.describe_security_groups()['SecurityGroups']
#     for security_group in security_groups:
#         if vpc_id == security_group['VpcId']:
#             security_group_id = security_group['GroupId']
#     print(security_group_id)    
# except:
#     pass



In [ ]:
# for sagemaker notebook
# try:
#     notebook_instance_name = sm.list_notebook_instances()['NotebookInstances'][0]['NotebookInstanceName']
#     notebook_instance = sm.describe_notebook_instance(NotebookInstanceName=notebook_instance_name)
#     security_group_id = notebook_instance['SecurityGroups'][0]
#     print(security_group_id)    
# except:
#     pass

## Create Redshift Cluster

In [ ]:
response = redshift.create_cluster(
        DBName=database_name,
        ClusterIdentifier=redshift_cluster_identifier,
        ClusterType=cluster_type,
        NodeType=node_type,
#         NumberOfNodes=int(number_nodes),       
        MasterUsername=master_user_name,
        MasterUserPassword=master_user_pw,
        IamRoles=[iam_role_redshift_arn],
        VpcSecurityGroupIds=[security_group_id],
        Port=5439,
        PubliclyAccessible=False
)

print(response)

# Please Wait for Cluster Status  `Available`

In [ ]:
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

In [ ]:
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
print(response)

In [ ]:
host = response['Clusters'][0]['Endpoint']['Address']
port = response['Clusters'][0]['Endpoint']['Port']

### Create Secret in Secrets Manager

AWS Secrets Manager is a service that enables you to easily rotate, manage, and retrieve database credentials, API keys, and other secrets throughout their lifecycle. Using Secrets Manager, you can secure and manage secrets used to access resources in the AWS Cloud, on third-party services, and on-premises.

In [ ]:
secretsmanager = boto3.client('secretsmanager')
secretstring = f'"username":"{master_user_name}","password":"{master_user_pw}","engine":"redshift", \
"host":"{host}","port": "{port}","dbClusterIdentifier":"{redshift_cluster_identifier}", "db":"{database_name}"'

secretstring 

In [ ]:
try:
    response = secretsmanager.create_secret(
        Name='bankdemo_redshift_login2',
        Description='BankDemo Redshift Login',
        SecretString= '{' + secretstring + '}',
        Tags=[
            {
                'Key': 'name',
                'Value': 'bankdemo_redshift_login'
            },
        ]
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceExistsException':
        print("Secret already exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [ ]:
# %%javascript

# try {
#     Jupyter.notebook.save_checkpoint();
#     Jupyter.notebook.session.delete();
# }
# catch(err) {
#     // NoOp
# }